# Markowitz Efficient Frontier

## 1. Imports

In [63]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm
from fetchData import fetch_raw_data_yf, get_matrices, getNasdaqStocks

## 2. Fetch Data

### Get all Nasdaq Stocks

In [64]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT"    # Lockheed Martin Corporation
]

assets

['AAPL',
 'MSFT',
 'AMZN',
 'GOOGL',
 'GOOG',
 'META',
 'TSLA',
 'UA',
 'JPM',
 'V',
 'JNJ',
 'WMT',
 'PG',
 'UNH',
 'MA',
 'NVDA',
 'HD',
 'BAC',
 'DIS',
 'PYPL',
 'VZ',
 'ADBE',
 'CMCSA',
 'NFLX',
 'KO',
 'NKE',
 'PFE',
 'MRK',
 'PEP',
 'T',
 'ABT',
 'CRM',
 'ORCL',
 'ABBV',
 'CSCO',
 'INTC',
 'TMO',
 'XOM',
 'ACN',
 'LLY',
 'COST',
 'MCD',
 'DHR',
 'MDT',
 'NEE',
 'BMY',
 'QCOM',
 'CVX',
 'WFC',
 'LMT']

In [65]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

[*********************100%***********************]  50 of 50 completed


Omitted assets ( 0 ):  []
Time to fetch data: 0.35 seconds


## 3. Mean, Volatility and Covariance

In [66]:
annualized_returns = get_matrices(raw_data, max_combination ,None)[0][2]
cov = get_matrices(raw_data, max_combination ,None)[0][1]
names = get_matrices(raw_data, max_combination ,None)[0][0]
volatility = np.sqrt(np.diag(cov)) #standard deviation

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

cov

,AAPL_adj_close,MSFT_adj_close,AMZN_adj_close,GOOGL_adj_close,GOOG_adj_close,META_adj_close,TSLA_adj_close,UA_adj_close,JPM_adj_close,V_adj_close,...,COST_adj_close,MCD_adj_close,DHR_adj_close,MDT_adj_close,NEE_adj_close,BMY_adj_close,QCOM_adj_close,CVX_adj_close,WFC_adj_close,LMT_adj_close
AAPL_adj_close,0.052350,0.025760,0.025213,0.021598,0.021686,0.025936,0.023558,0.006602,0.019057,0.019457,...,0.012517,0.011176,0.011747,0.013203,0.006910,0.013258,0.020975,0.014238,0.015651,0.011106
MSFT_adj_close,0.025760,0.050608,0.033346,0.028595,0.029584,0.027211,0.023770,0.013721,0.021409,0.024612,...,0.012159,0.015608,0.014713,0.015931,0.009246,0.013778,0.021582,0.017409,0.018728,0.012183
AMZN_adj_close,0.025213,0.033346,0.079122,0.037567,0.037850,0.038321,0.033361,0.006197,0.016572,0.023990,...,0.011680,0.015339,0.013700,0.014151,0.008331,0.012827,0.014579,0.012252,0.015441,0.007485
GOOGL_adj_close,0.021598,0.028595,0.037567,0.047776,0.047822,0.032502,0.024031,0.010336,0.015773,0.021277,...,0.011791,0.012865,0.011403,0.014629,0.007616,0.012039,0.014508,0.011065,0.013900,0.009658
GOOG_adj_close,0.021686,0.029584,0.037850,0.047822,0.048943,0.032390,0.023958,0.011479,0.015998,0.021201,...,0.011547,0.012941,0.011681,0.014811,0.007291,0.011939,0.014980,0.011335,0.014330,0.009841
META_adj_close,0.025936,0.027211,0.038321,0.032502,0.032390,0.057482,0.031729,0.009447,0.017084,0.023070,...,0.009995,0.014075,0.014296,0.014815,0.007990,0.013886,0.009083,0.012995,0.014234,0.010725
TSLA_adj_close,0.023558,0.023770,0.033361,0.024031,0.023958,0.031729,0.142595,0.017866,0.019142,0.022035,...,0.010310,0.011631,0.013591,0.019317,0.009471,0.015262,0.016221,0.017727,0.016006,0.008883
UA_adj_close,0.006602,0.013721,0.006197,0.010336,0.011479,0.009447,0.017866,0.218776,0.019450,0.012285,...,0.008769,0.003426,-0.005975,0.006837,-0.002951,0.004644,0.019090,0.008761,0.012788,-0.002039
JPM_adj_close,0.019057,0.021409,0.016572,0.015773,0.015998,0.017084,0.019142,0.019450,0.045418,0.020929,...,0.012368,0.012930,0.011586,0.014226,0.001909,0.015772,0.020347,0.024178,0.036454,0.012917
V_adj_close,0.019457,0.024612,0.023990,0.021277,0.021201,0.023070,0.022035,0.012285,0.020929,0.037212,...,0.010755,0.013035,0.015805,0.017576,0.008100,0.014695,0.018087,0.014446,0.019078,0.011203


In [67]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [68]:
portfolio_size = 5
number_of_portfolios = 1000

mean_variance_pairs = []
hover_texts = []

tickers_list = []
weights_list = []

for _ in range(number_of_portfolios):
    assets = np.random.choice(list(raw_data.columns), portfolio_size, replace=False)
    weights = np.random.rand(portfolio_size)
    weights /= weights.sum()

    portfolio_E_Return = 0
    portfolio_E_Variance = 0

    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
        for j in range(len(assets)):
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
    
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    tickers_list.append(assets)
    weights_list.append(weights)

    hover_text = f"Return: {portfolio_E_Return:.2%}<br>Volatility: {portfolio_E_Variance**0.5:.2f}<br>" + "<br>".join([f"{assets[i]}: {weights[i]:.2f}" for i in range(portfolio_size)])
    hover_texts.append(hover_text)

mean_variance_pairs = np.array(mean_variance_pairs)

In [69]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mean_variance_pairs[:, 1]**0.5,  # Volatility
    y=mean_variance_pairs[:, 0],  # Returns
    mode='markers',
    marker=dict(
        color=(mean_variance_pairs[:, 0]-risk_free_rate)/(mean_variance_pairs[:, 1]**0.5),  # Sharpe ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    text=hover_texts  # Use the prepared hover texts
))

fig.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig.show()

In [70]:
n_assets = 5
mean_variance_pairs = []
weights_list = []
tickers_list = []
hover_texts = [] 

for _ in tqdm(range(10000)):
    next_i = False
    while True:
        assets = np.random.choice(list(raw_data.columns), n_assets, replace=False)
        weights = np.random.rand(n_assets)
        weights /= weights.sum()

        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        # Check for dominated portfolios
        for R, V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break

        if next_i:
            break

        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        
        # Generate hover text for the current portfolio
        hover_text = "<br>".join([f"{assets[i]}: Weight={weights[i]:.2f}" for i in range(n_assets)])
        hover_texts.append(f"Return: {portfolio_E_Return:.2%}<br>Volatility: {portfolio_E_Variance**0.5:.2f}<br>{hover_text}")
        
        break

mean_variance_pairs = np.array(mean_variance_pairs)


100%|██████████| 10000/10000 [00:01<00:00, 6979.37it/s]


In [71]:
len(mean_variance_pairs)

127

In [72]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mean_variance_pairs[:, 1]**0.5,
    y=mean_variance_pairs[:, 0],
    mode='markers',
    marker=dict(
        color=(mean_variance_pairs[:, 0])/(mean_variance_pairs[:, 1]**0.5),
        showscale=True, 
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=hover_texts
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
)

fig.show()